## Imports

In [1]:
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import jupyros as jr
import rospy
import tf
import actionlib

from __future__ import print_function
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from IPython.display import clear_output
from geometry_msgs.msg import Twist
from std_msgs.msg import String
from std_srvs.srv import Empty
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseActionFeedback
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from actionlib_msgs.msg import GoalStatusArray
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
from matplotlib.animation import FuncAnimation

## Global variables, publishers and node initialization

In [2]:
#Initiliaze node
rospy.init_node('robot_controller')

# Initialize publishers
pub_vel = rospy.Publisher('/cmd_vel', Twist, queue_size=100)

# Goal variables
client_goal = actionlib.SimpleActionClient('move_base', MoveBaseAction)
client_goal.wait_for_server()
goal_set = False
goal_active = False
counts_reached = 0
counts_not_reached = 0
global start_time

# Velocity and collision assistant
vel_msg = Twist()
manual = False
assistant = False
threshold_collision = 0.6
global obstacles_range

# Menùs widgets
global main_menu
global automatic_mode
global manual_mode

## Functions

In [3]:
#cancels the goal by using the action client
def cancel_goal():
    global client_goal, goal_set
    if (goal_set):
        client_goal.cancel_goal()
        goal_set = False
        print("\n  ==   GOAL CANCELLED   == ")
    else:
        print("\n  ==   NO GOAL TO CANCEL   ==")

        
#clears the output and shows the menu given in input
def display_menu(menu):
    clear_output(wait=True)
    display(menu)

## Subscribers and callbacks

In [4]:
#callback for the laserscan: if assistant is on, it avoid the robot
#to touch walls by checking the walls distance and modifying the velocity msg
def driving_assistant(laser_msg):

    global vel_msg, threshold_collision, obstacles_range, manual
    
    #saves the laserscan ranges reversed in a global variable to be used to plot
    obstacles_range = laser_msg.ranges[::-1]
    
    if not manual:
        return
    
    # Control for the nearest walls if assistant is active
    if assistant:
        regions = {
            'right':  min(min(laser_msg.ranges[0:239]), 10),
            'front':  min(min(laser_msg.ranges[240:479]), 10),
            'left':   min(min(laser_msg.ranges[480:719]), 10),
        }

        # Set velocity to 0 if there is an obstacle near in the direction in which I'm going
        if ((regions['front'] < threshold_collision and vel_msg.linear.x > 0) 
            or (regions['right'] < threshold_collision and vel_msg.angular.z < 0) 
            or (regions['left'] < threshold_collision and vel_msg.angular.z > 0)):
            
            vel_msg.linear.x = 0
            vel_msg.angular.z = 0
            
    # Publish the velocity
    pub_vel.publish(vel_msg)

#callback for the feedback of the action: it check the state of the goal and
# the duration of the action, cancelling the goal if a timeout of 1 minut is reached
def check_status(status_msg):
    global start_time, counts_reached, counts_not_reached, goal_set
    
    if not goal_set:
        return
        
    # Timeout
    if (rospy.Time.now().to_sec() - start_time.to_sec()) > 60 :
        display_menu(main_menu)
        print("\n  ==   TIMEOUT EXCEEDED: GOAL CANNOT BE REACHED   == ")
        cancel_goal()
        counts_not_reached += 1
        return
    
    # SUCCEEDED
    if client_goal.get_state() == 3:
        display_menu(main_menu)
        print("\n  ==   GOAL REACHED   == ")
        counts_reached += 1
        goal_set = False

    
# Initialize subscribers
sub_laser = jr.subscribe('/scan', LaserScan, driving_assistant)
sub_status = jr.subscribe('move_base/status', GoalStatusArray, check_status)

## Main menu

In [5]:
b1 = Button(description='AUTOMATIC MODE - Set a goal to be reached autonomously',
    layout=Layout(width='715px', height ='100px', align="center", grid_area='b1'),
    style=ButtonStyle(button_color='LightSeaGreen'))

b2 = Button(description='MANUAL MODE - Control the robot with buttons',
    layout=Layout(width='auto', height ='100px', align="center", grid_area='b2'),
    style=ButtonStyle(button_color='lightskyblue'))

b3 = Button(description='MANUAL MODE + Assistant to avoid obstacles',
    layout=Layout(width='auto', height ='100px',align="center", grid_area='b3'),
    style=ButtonStyle(button_color='lightblue'))

b4 = Button(description='CANCEL GOAL',
    layout=Layout(width='auto',height ='50px', align="center", grid_area='b4'),
    style=ButtonStyle(button_color='salmon'))

main_menu = GridBox(children=[b1, b2, b3, b4],
    layout=Layout(
        width='720px',
        grid_template_rows='auto',
        grid_template_columns='50% 50%',
        grid_template_areas='''
        "b1 ."
        "b2 b3"
        ". b4"
        ''')
    )

def clicked_manual(b):
    global assistant, manual
    manual = True
    display_menu(manual_mode)
    if b is b3:
        assistant = True
        print('\n  ==   Collision Assistant ON   ==')
    else:
        print('\n  ==   Collision Assistant OFF   ==')

def clicked_auto(b):
    display_menu(automatic_mode)

def clicked_cancel(b):
    cancel_goal()
        
b1.on_click(clicked_auto)
b2.on_click(clicked_manual)
b3.on_click(clicked_manual)
b4.on_click(clicked_cancel)

## Automatic mode menu

In [6]:
x_goal = widgets.FloatSlider(
        value=0,
        min=-10,
        max=10,
        step=0.1,
        description='X:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='500px'))

y_goal = widgets.FloatSlider(
        value=0,
        min=-10,
        max=10,
        step=0.1,
        description='Y:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='500px'))

send_button = Button(description='SEND GOAL',
    layout=Layout(width='200px', height ='30px', align="center", grid_area='send_button'),
    style=ButtonStyle(button_color='LightSeaGreen'))

back_button1 = Button(description='BACK TO MAIN MENU',
    layout=Layout(width='200px', height ='30px', align="center", grid_area='send_button'),
    style=ButtonStyle(button_color='silver'))

automatic_mode = HBox([VBox([x_goal, y_goal]), VBox([send_button, back_button1])])

def clicked_send_goal(b):
    global goal_set, client_goal, start_time
    # Initialize the goal
    goal_msg = MoveBaseGoal()
    goal_msg.target_pose.header.frame_id = "map"
    goal_msg.target_pose.header.stamp = rospy.Time.now()
    goal_msg.target_pose.pose.orientation.w = 1
    goal_msg.target_pose.pose.position.x = x_goal.value
    goal_msg.target_pose.pose.position.y = y_goal.value
    
    # Use the goal action
    start_time = rospy.Time.now()
    goal_set = True
    client_goal.send_goal(goal_msg)
    display_menu(main_menu)
    print("\n  ==   GOAL PUBLISHED   == ")
    
    
def clicked_back_button(b):
    global manual
    if manual:
        manual = False
    display_menu(main_menu)
    
back_button1.on_click(clicked_back_button)
send_button.on_click(clicked_send_goal)


## Manual mode menu

In [7]:
#↗↖↘↙→←↑↓

up_left = Button(description='↖↖↖',
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up_left'),
    style=ButtonStyle(button_color='LightSeaGreen'))

up = Button(description='↑↑↑',
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up'),
    style=ButtonStyle(button_color='LightSeaGreen'))

up_right = Button(description='↗↗↗',                  
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up_right'),
    style=ButtonStyle(button_color='LightSeaGreen'))

left = Button(description='←←←',
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up_left'),
    style=ButtonStyle(button_color='LightSeaGreen'))

stop = Button(description='STOP',
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up_left'),
    style=ButtonStyle(button_color='salmon'))

right = Button(description='→→→',
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up'),
    style=ButtonStyle(button_color='LightSeaGreen'))

down_left = Button(description='↙↙↙',
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up'),
    style=ButtonStyle(button_color='LightSeaGreen'))

down = Button(description='↓↓↓',
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up'),
    style=ButtonStyle(button_color='LightSeaGreen'))

down_right = Button(description='↘↘↘',                  
    layout=Layout(width='80px', height ='80px', align="center", grid_area='up_right'),
    style=ButtonStyle(button_color='LightSeaGreen'))

back_button2 = Button(description='BACK TO MAIN MENU',                  
    layout=Layout(width='200px', height ='80px', align="center", grid_area='up_right'),
    style=ButtonStyle(button_color='silver'))

manual_mode = VBox([HBox([up_left, up, up_right]),
                    HBox([left, stop , right, back_button2]),
                    HBox([down_left, down , down_right])])
    
def clicked_direction(b):
    global vel_msg
    # Up/down
    if b == up or b == up_left or b == up_right:
        vel_msg.linear.x = 0.5
    elif b == down or b == down_left or b == down_right:
        vel_msg.linear.x = -0.5
    else:
        vel_msg.linear.x = 0
    # Right/left
    if b == up_left or b == left or b == down_right:
        vel_msg.angular.z = 0.5
    elif b == up_right or b == right or b == down_left:
        vel_msg.angular.z = -0.5
    else:
        vel_msg.angular.z = 0
    # Stop
    if b == stop:
        vel_msg.linear.x = 0
        vel_msg.angular.z = 0

up.on_click(clicked_direction)
up_left.on_click(clicked_direction)
up_right.on_click(clicked_direction)
left.on_click(clicked_direction)
stop.on_click(clicked_direction)
right.on_click(clicked_direction)
down_left.on_click(clicked_direction)
down.on_click(clicked_direction)
down_right.on_click(clicked_direction)

back_button2.on_click(clicked_back_button)

## Plots

In [8]:
%matplotlib widget

############ LASER SCAN

class VisualiserScan:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = self.ax.plot([], [], 'ro')
        self.x_data = np.linspace(-90,90,720).tolist() 
        
    def plot_init(self):
        self.ax.set_xlim(-90, 90)
        self.ax.set_xlabel('Radius')
        self.ax.set_xticks([-90, 0, 90])
        self.ax.set_ylim(0, 15)
        self.ax.set_ylabel('Distance')
        self.ax.set_title("Laser scan")
        return self.ln
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, obstacles_range)
        return self.ln

vis_scan = VisualiserScan()


############ ROBOT POSITION

class VisualiserOdom:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = self.ax.plot([], [], 'gx')
        self.x_data, self.y_data = [] , []
        
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_xlabel('X')
        self.ax.set_ylim(-10, 10)
        self.ax.set_ylabel('Y')
        self.ax.set_title("Odometry of the robot")
        return self.ln

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
vis_odom = VisualiserOdom()
sub_odom = jr.subscribe('/odom', Odometry, vis_odom.odom_callback)


############# REACHED/UNREACHED TARGETS

fig_bar, ax_bar = plt.subplots()
ax_bar.set_title("Reached/Unreached targets count")

y1,y2, = [], []
def update_bar(curr):
    global counts_reached, counts_not_reached
    y1 = counts_reached
    y2 = counts_not_reached
    ax_bar.bar(["Reached targets", "Unreached targets"], [y1,y2], color = ['green', 'red'])

############# ANIMATIONS
    
ani_odom = FuncAnimation(vis_odom.fig, vis_odom.update_plot, init_func=vis_odom.plot_init)
ani_scan = FuncAnimation(vis_scan.fig, vis_scan.update_plot, init_func=vis_scan.plot_init)
ani_targets = FuncAnimation(fig_bar, update_bar)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## User Interface

In [9]:
output = widgets.Output()
display(main_menu, output)


  ==   Collision Assistant ON   ==
